# Chapter 6: Decision Trees 

## 6.1 Training and Visualizing a Decision Tree

## 6.2 Making Predictions

## 6.3 Estimating Class Probabilities

## 6.4 The CART Training Algorithm

## 6.5 Computational Complexity

## 6.6 Gini Impurity or Entropy?

## 6.7 Regularization Hyperparameters

## 6.8 Regression

## 6.9 Instability